In [1]:
import os
import pandas as pd
import numpy as np

import sqlalchemy
from sqlalchemy import create_engine

In [2]:
engine = create_engine('sqlite:///DisasterResponse_try3.db')
df = pd.read_sql_table("Message", engine)
print(f"{type(df)},\n{len(df)}")


<class 'pandas.core.frame.DataFrame'>,
26216


In [6]:
df.head(1)

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [3]:
df.columns

Index(['id', 'message', 'original', 'genre', 'related', 'request', 'offer',
       'aid_related', 'medical_help', 'medical_products', 'search_and_rescue',
       'security', 'military', 'child_alone', 'water', 'food', 'shelter',
       'clothing', 'money', 'missing_people', 'refugees', 'death', 'other_aid',
       'infrastructure_related', 'transport', 'buildings', 'electricity',
       'tools', 'hospitals', 'shops', 'aid_centers', 'other_infrastructure',
       'weather_related', 'floods', 'storm', 'fire', 'earthquake', 'cold',
       'other_weather', 'direct_report'],
      dtype='object')

In [5]:
# extract data for ml model
X = df['message']
# drop additional cols
y = df.drop(['id', 'message', 'original', 'genre'], axis=1)
print(X.shape, y.shape)

(26216,) (26216, 36)
